In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
from gensim import corpora, models, similarities
import re

In [6]:
STOP_WORD_FILE = 'vietnamese-stopwords.txt'

In [7]:
with open(STOP_WORD_FILE, 'r', encoding='utf-8') as file:
    stop_words = file.read()

stop_words = stop_words.split('\n')

### underthesea

In [8]:
data = ["Kem Dưỡng Da Ban Ngày Trắng Hồng Rạng Rỡ Pond's White Beauty 50g với công thức phức hợp dưỡng trắng Vitamin B3 giúp tăng cường khả năng dưỡng trắng từ sâu bên trong, làm mờ vết thâm sạm, cho làn da trắng hồng rạng rỡ bên ngoài.",
        "Kem Dưỡng Da Đêm Ngày Trắng Hồng Rạng Rỡ Pond's White Beauty 50g với công thức phức hợp dưỡng trắng Vitamin B3+ giúp tăng cường khả năng dưỡng trắng da liên tục suốt cả đêm từ sâu bên trong, mang đến làn da sáng mịn rạng rỡ sau mỗi lần sử dụng.",
        "Kem Dưỡng Trắng Da Ban Đêm Senka White Beauty Glow Gel Cream 50g Với công nghệ độc quyền và tiên tiến Natu-Ence từ Shiseido giúp nâng cao hiệu quả của các dưỡng chất từ thiên nhiên, Kem dưỡng trắng da ban đêm Senka sẽ bổ sung các dưỡng chất cần thiết và độ ẩm cho làn da luôn mịn màng, dưỡng trắng da vượt trội.",
        "Kem dưỡng nâng tông da trắng hồng tự nhiên Laneige White Dew Tone Up Cream 50ml giúp làm sáng ngay lập tức vùng da bị vàng và xỉn màu với kết cấu giống dạng sữa dưỡng giàu độ ẩm. Công thức sữa dưỡng dầu/nước chứa TiO2 (titanium dioxide) mang đến hiệu quả làm tăng tông màu da cho những vùng da bị tối hay xỉn màu",
        "Kem Dưỡng Nivea Ngọc Trai Làm Sáng Da Ban Đêm 50ml 5 in 1 Pearl Filler Pearl White Night Face Cream Công thức vượt trội được chiết xuất từ các dưỡng chất thiên nhiên, vừa giúp dưỡng trắng da, ngay cả làn da sạm đen, vừa nuôi dưỡng làn da trong lúc ngủ. Cho bạn làn da trắng mượt như nước, không ngại bắt nắng",
        "Phấn Phủ Dưỡng Ẩm Dạng Nén Laneige Light Fit Pact 9.5g. Kết cấu mỏng nhẹ, độ bám dính cao Giúp che phủ hiệu quả hoàn hảo Thể hiện màu sắc tự nhiên",
        "Phấn Phủ Kiềm Dầu Dạng Bột Khoáng Innisfree No Sebum Mineral Powder 5g dạng bột có chiết xuất 100% từ bạc hà và hạt ngọc trai giúp kiềm hút dầu và loại bỏ bã nhờn dư thừa trên da mang đến làn da sáng mịn.", 
        "Sữa Rửa Mặt Ngăn Ngừa Lão Hóa Pond's Age Miracle 100g gồm 6 dưỡng chất chuyên biệt, giúp thúc đẩy quá trình tái tạo da đẩy lùi quá trình lão hóa.", 
        "Sữa Rửa Mặt Ngăn Ngừa Lão Hóa Da Innisfree Jeju Pomegranate Revitalizing Foam Cleanser 150ml từ nước ép lựu và dầu hạt lựu giúp làm sạch da sâu da mà không gây khô căng."
        ]

In [9]:
df = pd.DataFrame(data, columns = ['products'])

In [10]:
df.head()

,products
0,Kem Dưỡng Da Ban Ngày Trắng Hồng Rạng Rỡ Pond'...
1,Kem Dưỡng Da Đêm Ngày Trắng Hồng Rạng Rỡ Pond'...
2,Kem Dưỡng Trắng Da Ban Đêm Senka White Beauty ...
3,Kem dưỡng nâng tông da trắng hồng tự nhiên Lan...
4,Kem Dưỡng Nivea Ngọc Trai Làm Sáng Da Ban Đêm ...


In [11]:
# word_tokenize
df["products_wt"]=df["products"].apply(lambda x: word_tokenize(x, format="text"))

In [12]:
df.head()

,products,products_wt
0,Kem Dưỡng Da Ban Ngày Trắng Hồng Rạng Rỡ Pond'...,Kem Dưỡng_Da Ban_Ngày Trắng_Hồng Rạng_Rỡ Pond'...
1,Kem Dưỡng Da Đêm Ngày Trắng Hồng Rạng Rỡ Pond'...,Kem Dưỡng_Da_Đêm Ngày Trắng_Hồng Rạng_Rỡ Pond'...
2,Kem Dưỡng Trắng Da Ban Đêm Senka White Beauty ...,Kem Dưỡng_Trắng Da_Ban_Đêm Senka White_Beauty ...
3,Kem dưỡng nâng tông da trắng hồng tự nhiên Lan...,Kem dưỡng nâng tông da trắng hồng tự_nhiên Lan...
4,Kem Dưỡng Nivea Ngọc Trai Làm Sáng Da Ban Đêm ...,Kem Dưỡng_Nivea Ngọc_Trai Làm_Sáng Da_Ban Đêm ...


### genshim

In [13]:
# Tokenize(split) the sentences into words
products_gem = [[text for text in x.split()] for x in df.products_wt]

In [14]:
len(products_gem)

9

In [15]:
products_gem[:1]

[['Kem',
  'Dưỡng_Da',
  'Ban_Ngày',
  'Trắng_Hồng',
  'Rạng_Rỡ',
  "Pond's_White",
  'Beauty',
  '50',
  'g',
  'với',
  'công_thức',
  'phức_hợp_dưỡng',
  'trắng',
  'Vitamin_B3',
  'giúp',
  'tăng_cường',
  'khả_năng',
  'dưỡng_trắng',
  'từ',
  'sâu',
  'bên',
  'trong',
  ',',
  'làm',
  'mờ',
  'vết',
  'thâm_sạm',
  ',',
  'cho',
  'làn',
  'da',
  'trắng',
  'hồng',
  'rạng_rỡ',
  'bên',
  'ngoài',
  '.']]

In [16]:
# phần này viết thành function, bỏ bớt những từ ko cần thiết
products_gem_re = [[re.sub('[0-9]+','', e) for e in text] for text in products_gem] # số
products_gem_re = [[t.lower() for t in text if not t in ['', ' ', ',', '.', '...', '-',':', ';', '?', '%', '(', ')', '+', '/', 'g', 'ml']] for text in  products_gem_re] # ký tự đặc biệt
products_gem_re = [[t for t in text if not t in stop_words] for text in products_gem_re] # stopword
# ...

In [17]:
dictionary = corpora.Dictionary(products_gem_re)

In [18]:
dictionary.token2id

{'ban_ngày': 0,
 'beauty': 1,
 'công_thức': 2,
 'da': 3,
 'dưỡng_da': 4,
 'dưỡng_trắng': 5,
 'giúp': 6,
 'hồng': 7,
 'kem': 8,
 'khả_năng': 9,
 'làn': 10,
 'mờ': 11,
 'phức_hợp_dưỡng': 12,
 "pond's_white": 13,
 'rạng_rỡ': 14,
 'sâu': 15,
 'thâm_sạm': 16,
 'trắng': 17,
 'trắng_hồng': 18,
 'tăng_cường': 19,
 'vitamin_b': 20,
 'vết': 21,
 'dưỡng_da_đêm': 22,
 'liên_tục': 23,
 'mịn': 24,
 'suốt': 25,
 'đêm': 26,
 'ban_đêm': 27,
 'bổ_sung': 28,
 'cream': 29,
 'công_nghệ': 30,
 'cần_thiết': 31,
 'da_ban_đêm': 32,
 'dưỡng': 33,
 'dưỡng_chất': 34,
 'glow_gel': 35,
 'hiệu_quả': 36,
 'kem_dưỡng': 37,
 'mịn_màng': 38,
 'natu-ence': 39,
 'nâng': 40,
 'senka': 41,
 'shiseido': 42,
 'thiên_nhiên': 43,
 'tiên_tiến': 44,
 'vượt_trội': 45,
 'white_beauty': 46,
 'độ_ẩm': 47,
 'độc_quyền': 48,
 'chứa': 49,
 'dew_tone': 50,
 'dạng': 51,
 'dầu': 52,
 'giàu': 53,
 'kết_cấu': 54,
 'laneige_white': 55,
 'màu': 56,
 'màu_da': 57,
 'sữa_dưỡng': 58,
 'tio': 59,
 'titanium_dioxide': 60,
 'tông': 61,
 'tối': 62,
 

In [20]:
# Numbers of features (word) in dictionary
feature_cnt = len(dictionary.token2id)
feature_cnt

140

In [21]:
# Obtain corpus based on dictionary (dense matrix)
corpus = [dictionary.doc2bow(text) for text in products_gem_re]

In [24]:
corpus[0] # id và số lần xuất hiện của token trong văn bản

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 2),
 (15, 1),
 (16, 1),
 (17, 2),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 1)]

In [25]:
# Use TF-IDF Model to process corpus, obtaining index
tfidf = models.TfidfModel(corpus)
# tính toán sự tương tự trong ma trận thưa thớt
index = similarities.SparseMatrixSimilarity(tfidf[corpus],
                                            num_features = feature_cnt)

In [27]:

product_selection = df.head(1)
product_selection

,products,products_wt
0,Kem Dưỡng Da Ban Ngày Trắng Hồng Rạng Rỡ Pond'...,Kem Dưỡng_Da Ban_Ngày Trắng_Hồng Rạng_Rỡ Pond'...


In [28]:
# sản phẩm đang xem
name_description_pre = product_selection['products_wt'].to_string(index=False)

In [29]:
# can phai goi function chuan hoa da ap dung cho cac san pham
view_product = name_description_pre.lower().split()

In [30]:
# Convert search words into Sparse Vectors
# nhớ xài cái dictionary cũ
kw_vector = dictionary.doc2bow(view_product)

In [31]:
kw_vector

[(0, 1), (4, 1), (8, 1), (14, 1), (18, 1)]

In [32]:
# similarity calculation
sim = index[tfidf[kw_vector]]

In [33]:
# print result
for i in range(len(sim)):
    # Vì lấy mẫu đầu tiên để xem nên bỏ qua mẫu đầu tiên
    print('keyword is similar to doc_index %d: %.2f' % (i, sim[i]))

keyword is similar to doc_index 0: 0.59
keyword is similar to doc_index 1: 0.27
keyword is similar to doc_index 2: 0.01
keyword is similar to doc_index 3: 0.01
keyword is similar to doc_index 4: 0.01
keyword is similar to doc_index 5: 0.00
keyword is similar to doc_index 6: 0.00
keyword is similar to doc_index 7: 0.00
keyword is similar to doc_index 8: 0.00


In [34]:
# Giả sử sản phẩm liên quan nhất có index là 1 như trên
product_famillier = df.iloc[[1]]

In [35]:
product_famillier

,products,products_wt
1,Kem Dưỡng Da Đêm Ngày Trắng Hồng Rạng Rỡ Pond'...,Kem Dưỡng_Da_Đêm Ngày Trắng_Hồng Rạng_Rỡ Pond'...


### cosine_similarity

In [36]:
# mẫu công thức toán 
from numpy import dot
from numpy.linalg import norm
X = [1,2]
Y = [2,2]
cos_sim = dot(X,Y) / (norm(X)*norm(Y))
print(cos_sim)

0.9486832980505138


In [ ]:
# vẫn thực hiện các bước như bên trên, sau đó mới vào tfidf

In [37]:
tf = TfidfVectorizer(analyzer='word', min_df=0, stop_words=stop_words)

In [38]:
tfidf_matrix = tf.fit_transform(df.products_wt)

In [39]:
cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [40]:
cosine_similarities

array([[1.        , 0.69519827, 0.1551208 , 0.12374782, 0.15994914,
        0.00965768, 0.04347849, 0.04908612, 0.05106241],
       [0.69519827, 1.        , 0.15715171, 0.10934188, 0.21851252,
        0.01015654, 0.10185011, 0.06193459, 0.0751564 ],
       [0.1551208 , 0.15715171, 1.        , 0.19403449, 0.26562584,
        0.0304808 , 0.06240797, 0.0773547 , 0.04822855],
       [0.12374782, 0.10934188, 0.19403449, 1.        , 0.1497956 ,
        0.08191914, 0.08858269, 0.0278274 , 0.08692276],
       [0.15994914, 0.21851252, 0.26562584, 0.1497956 , 1.        ,
        0.00771569, 0.13257418, 0.06271941, 0.12209577],
       [0.00965768, 0.01015654, 0.0304808 , 0.08191914, 0.00771569,
        1.        , 0.04378038, 0.00966135, 0.        ],
       [0.04347849, 0.10185011, 0.06240797, 0.08858269, 0.13257418,
        0.04378038, 1.        , 0.06232388, 0.10206116],
       [0.04908612, 0.06193459, 0.0773547 , 0.0278274 , 0.06271941,
        0.00966135, 0.06232388, 1.        , 0.14212572],


In [41]:
df_show = pd.DataFrame(cosine_similarities)
df_show.head(9)

,0,1,2,3,4,5,6,7,8
0,1.000000,0.695198,0.155121,0.123748,0.159949,0.009658,0.043478,0.049086,0.051062
1,0.695198,1.000000,0.157152,0.109342,0.218513,0.010157,0.101850,0.061935,0.075156
2,0.155121,0.157152,1.000000,0.194034,0.265626,0.030481,0.062408,0.077355,0.048229
3,0.123748,0.109342,0.194034,1.000000,0.149796,0.081919,0.088583,0.027827,0.086923
4,0.159949,0.218513,0.265626,0.149796,1.000000,0.007716,0.132574,0.062719,0.122096
5,0.009658,0.010157,0.030481,0.081919,0.007716,1.000000,0.043780,0.009661,0.000000
6,0.043478,0.101850,0.062408,0.088583,0.132574,0.043780,1.000000,0.062324,0.102061
7,0.049086,0.061935,0.077355,0.027827,0.062719,0.009661,0.062324,1.000000,0.142126
8,0.051062,0.075156,0.048229,0.086923,0.122096,0.000000,0.102061,0.142126,1.000000


In [42]:
df_show.style.applymap(lambda x: "background-color: lightblue" if x>=1.0 else "background-color: green" if x>=0.2 and x<1.0 else "background-color: white")

,0,1,2,3,4,5,6,7,8
0,1.000000,0.695198,0.155121,0.123748,0.159949,0.009658,0.043478,0.049086,0.051062
1,0.695198,1.000000,0.157152,0.109342,0.218513,0.010157,0.101850,0.061935,0.075156
2,0.155121,0.157152,1.000000,0.194034,0.265626,0.030481,0.062408,0.077355,0.048229
3,0.123748,0.109342,0.194034,1.000000,0.149796,0.081919,0.088583,0.027827,0.086923
4,0.159949,0.218513,0.265626,0.149796,1.000000,0.007716,0.132574,0.062719,0.122096
5,0.009658,0.010157,0.030481,0.081919,0.007716,1.000000,0.043780,0.009661,0.000000
6,0.043478,0.101850,0.062408,0.088583,0.132574,0.043780,1.000000,0.062324,0.102061
7,0.049086,0.061935,0.077355,0.027827,0.062719,0.009661,0.062324,1.000000,0.142126
8,0.051062,0.075156,0.048229,0.086923,0.122096,0.000000,0.102061,0.142126,1.000000


In [43]:
# với mỗi sản phẩm, lấy 3 sản phẩm tương quan nhất
# function lay x san pham tuong quan nhat
# def find_highest_score(cosine_similarities, nums = 3):
results = {}

for idx, row in df.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[-4:-1]
    similar_items = [(cosine_similarities[idx][i]) for i in similar_indices]
    similar_items = [(cosine_similarities[idx][i], df.index[i]) for i in similar_indices]
    print(similar_items[0:])
    results[idx] = similar_items[0:]

[(0.15512079719991662, 2), (0.15994914305652225, 4), (0.6951982684968354, 1)]
[(0.15715171135957345, 2), (0.218512522363034, 4), (0.6951982684968354, 0)]
[(0.15715171135957345, 1), (0.19403448802686324, 3), (0.26562584202409356, 4)]
[(0.12374782171555312, 0), (0.14979560104276052, 4), (0.19403448802686324, 2)]
[(0.15994914305652225, 0), (0.218512522363034, 1), (0.26562584202409356, 2)]
[(0.03048080117011501, 2), (0.043780376453166577, 6), (0.08191914396190085, 3)]
[(0.10185010798524011, 1), (0.1020611638551705, 8), (0.13257417789271964, 4)]
[(0.06271940602042296, 4), (0.07735469936250719, 2), (0.14212571768388169, 8)]
[(0.1020611638551705, 6), (0.12209577477360442, 4), (0.14212571768388169, 7)]


In [44]:
# người dùng chọn sản phẩm -> lấy đc id sp -> dò vào ma trận kết quả theo id -> ra đc các id của sản phẩm liên quan -> show sp liên quan
results[0]

[(0.15512079719991662, 2), (0.15994914305652225, 4), (0.6951982684968354, 1)]